In [177]:
import glob
import tifffile
import numpy as np
import lightgbm as lgb
import warnings
import tqdm
import cv2
import os
import random
from tqdm import tqdm 
import natsort
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pylab as plt
import zipfile

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [178]:

train_path =  './train/s2_image/'
mask_path = './train/mask/'

masks = glob.glob(f'{mask_path}/*')
trains = glob.glob(f'{train_path}/*')
masks.sort()
trains.sort()

In [179]:
X = []
y = []
g = []

for i, (t, m) in tqdm(enumerate(zip(trains, masks))):
    img = tifffile.imread(t).astype(np.float)  #(23,24,12) 
    mask = tifffile.imread(m).astype(np.float)  #(23, 24)
    X.append(img.reshape(-1,12)) #(552,12)
    y.append(mask.reshape(-1))
    g.append(np.ones_like(mask.reshape(-1))*i) 
    
X = np.vstack(X) #(1133572, 12)
y = np.hstack(y)  #(1133572,)
g = np.hstack(g)  #(1133572,)

2066it [00:04, 512.17it/s]


In [181]:
temp = pd.DataFrame(X)
temp

,0,1,2,3,4,5,6,7,8,9,10,11
0,265.800583,279.066472,488.098252,331.572300,817.002914,2400.280728,3084.914212,3236.613360,3332.798479,3251.691757,1598.907575,818.413402
1,265.800583,271.067287,472.501621,322.083156,817.002914,2400.280728,3084.914212,3188.111508,3332.798479,3251.691757,1598.907575,818.413402
2,265.800583,240.942322,424.775329,279.931718,784.908433,2322.216009,2998.675982,2906.248048,3251.978778,3251.691757,1549.296809,798.364396
3,265.800583,222.820502,411.740847,245.060190,737.346930,2206.530216,2870.877686,2640.218320,3132.210318,3251.691757,1475.777544,768.653340
4,378.735906,229.331874,434.090056,254.714547,782.884060,2226.587592,2840.185718,2651.574272,3084.218565,3341.757199,1562.631939,809.818550
...,...,...,...,...,...,...,...,...,...,...,...,...
1133567,1166.000000,1049.347260,1190.897532,1254.509945,1365.000000,1379.000000,1468.000000,1463.018392,1461.000000,1490.000000,2027.000000,1859.000000
1133568,1166.000000,1090.473893,1243.402564,1362.630769,1365.000000,1379.000000,1468.000000,1572.583217,1461.000000,1490.000000,2027.000000,1859.000000
1133569,1089.344375,1097.961592,1251.727616,1360.179232,1399.362866,1489.137392,1583.130287,1621.304781,1591.402672,1639.199454,2009.671717,1797.029361
1133570,905.000000,1018.380712,1222.829764,1265.576354,1482.000000,1754.000000,1860.000000,1952.131222,1905.000000,1998.000000,1968.000000,1648.000000


In [182]:
temp['NDVIA'] = (temp[7]-temp[3])/(temp[7]+temp[3])
X = np.array(temp)

In [186]:
X.shape

(1133572, 13)

In [180]:
from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [125]:
lgb_params = {    
    'boosting_type':'gbdt',
    'num_leaves':31,
    'max_depth':-1,
    'n_estimators':1000,
    'random_state':32,
    }

In [136]:
from sklearn.metrics import f1_score

def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [157]:
xgb_params = {    
    'boosting_type':'gbdt',
    'num_leaves':31,
    'max_depth':4,
    'n_estimators':1000,
    'random_state':32,
    }

In [158]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [159]:

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
gkfold = GroupKFold(n_splits=4)
models = []
for i, (train_idx, valid_idx) in enumerate(gkfold.split(X, y, g)):
    train_x = X[train_idx] #(850411, 12)
    train_y = y[train_idx] #(850410,)
    
    val_x = X[valid_idx]  #(283162, 12)
    val_y = y[valid_idx]   #(283162,)
    
    # m = LGBMClassifier(**lgb_params, verbose=1, early_stopping_rounds=300)
    # m.fit(train_x, train_y, 
    #      eval_metric='f1',
    #      eval_set=[(val_x, val_y)],
    #      callbacks=[
    #         lgb.log_evaluation(100),
    #     ],
    # )
    m=XGBClassifier(**xgb_params,verbose=1) 
    m.fit(train_x,train_y,eval_set=[(val_x,val_y)])

    models.append(m)

[0]	validation_0-logloss:0.24753
[1]	validation_0-logloss:0.20914
[2]	validation_0-logloss:0.18456
[3]	validation_0-logloss:0.16735
[4]	validation_0-logloss:0.15522
[5]	validation_0-logloss:0.14658
[6]	validation_0-logloss:0.14020
[7]	validation_0-logloss:0.13345
[8]	validation_0-logloss:0.12922
[9]	validation_0-logloss:0.12563
[10]	validation_0-logloss:0.12141
[11]	validation_0-logloss:0.11740
[12]	validation_0-logloss:0.11556
[13]	validation_0-logloss:0.11346
[14]	validation_0-logloss:0.11142
[15]	validation_0-logloss:0.10976
[16]	validation_0-logloss:0.10852
[17]	validation_0-logloss:0.10705
[18]	validation_0-logloss:0.10606
[19]	validation_0-logloss:0.10467
[20]	validation_0-logloss:0.10423
[21]	validation_0-logloss:0.10344
[22]	validation_0-logloss:0.10286
[23]	validation_0-logloss:0.10207
[24]	validation_0-logloss:0.10126
[25]	validation_0-logloss:0.10084
[26]	validation_0-logloss:0.10027
[27]	validation_0-logloss:0.09989
[28]	validation_0-logloss:0.09941
[29]	validation_0-loglos

In [176]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# 3장 내용
def get_clf_eval(y_test, pred):
   confusion = confusion_matrix(y_test, pred)
   accuracy = accuracy_score(y_test, pred)
   precision = precision_score(y_test, pred)
   recall = recall_score(y_test, pred)
   f1 = f1_score(y_test, pred)
   print('오차 행렬')
   print(confusion)
   print('정확도 : {:.4f}, 정밀도 : {:.4f}, 재현율 : {:.4f},\
   F1 : {:.4f}'.format(accuracy,precision,recall,f1))

X = val_x.reshape(-1, 12)
preds = models[3].predict_proba(X)  
pred_mask = np.argmax(preds, axis=1).astype(np.uint8)

get_clf_eval(val_y, pred_mask)

오차 행렬
[[251770   4688]
 [  5885  20818]]
정확도 : 0.9627, 정밀도 : 0.8162, 재현율 : 0.7796,   F1 : 0.7975


In [115]:
test_path =  './evaluation/'
test_mask_path = './sample/'

masks = glob.glob(f'{test_mask_path}/*')
tests = glob.glob(f'{test_path}/*')
masks.sort()
tests.sort()

In [116]:
output_folder_path = 'output'
os.makedirs(output_folder_path, exist_ok=True)

for i, (m, t) in tqdm(enumerate(zip(masks, tests))):
    basename = os.path.basename(m)
    output_file = f'output/{basename}'
    
    img = tifffile.imread(t).astype(np.float)
    mask = tifffile.imread(m).astype(np.float)
    
    X = img.reshape(-1, 12)  
    shape_mask = mask.shape
    
    pred = 0
    for model in models:
        pred += model.predict_proba(X) / len(models)

    pred_mask = np.argmax(pred, axis=1).astype(np.uint8)
    pred_mask = pred_mask.reshape(shape_mask[0], shape_mask[1])
    
    tifffile.imwrite(output_file, pred_mask)

with zipfile.ZipFile('output_zip.zip', 'w') as zipf:
    for root, dirs, files in os.walk(output_folder_path):
        for file in files:
            zipf.write(os.path.join(root, file), file)

2066it [00:21, 96.53it/s] 
